# Analyses SeqApiPop : RFMix

- Mise en phase avec les SNP
    - Sélectionner les SNP
    - VCFs par Chromosme
    - Phaser avec Shapeit
        - Phasage
        - Convertir le génotype phasé en vcf
        - Concaténation des VCFs
        
- RFMix
- Sélectionner les échantillons de référence et d'interrogation
    - Créer des fichiers vcf
- Créer des cartes génétiques
    - Trouver le crossing-over dans les données de Liu et al. 2015
- Exécuter RFMix
    - Impossible de faire fonctionner le chromosome 1.
    
    
### Mise en phase avec shapeit

### Sélectionner les SNP

- MAF > 1
- Max allèles = 2
- Chromosomes en tant que nombres



Tout d'abord, les noms des chromosomes doivent être changés en nombres de 1 à 16 car nous n'incluons pas le chromosome mitochondrial

voici le script "ChangeChr" : pour changer les noms des chromosomes en numéros :


```bash
#!/bin/sh


module load bioinfo/bcftools-1.9



VCF=MetaGenotypesCalled403_raw_snps_filtre_isec_plink.vcf.gz


echo "NC_037638.1 1" >> chr_name_conv.txt
echo "NC_037639.1 2" >> chr_name_conv.txt
echo "NC_037640.1 3" >> chr_name_conv.txt
echo "NC_037641.1 4" >> chr_name_conv.txt
echo "NC_037642.1 5" >> chr_name_conv.txt
echo "NC_037643.1 6" >> chr_name_conv.txt
echo "NC_037644.1 7" >> chr_name_conv.txt
echo "NC_037645.1 8" >> chr_name_conv.txt
echo "NC_037646.1 9" >> chr_name_conv.txt
echo "NC_037647.1 10" >> chr_name_conv.txt
echo "NC_037648.1 11" >> chr_name_conv.txt
echo "NC_037649.1 12" >> chr_name_conv.txt
echo "NC_037650.1 13" >> chr_name_conv.txt
echo "NC_037651.1 14" >> chr_name_conv.txt
echo "NC_037652.1 15" >> chr_name_conv.txt
echo "NC_037653.1 16" >> chr_name_conv.txt
echo "NC_001566.1 17" >> chr_name_conv.txt


bcftools annotate --rename-chrs chr_name_conv.txt $VCF | bgzip > rename.vcf.gz

bcftools index rename.vcf.gz
```


Ici, le script pour la sélection des SNP :

```bash
#!/bin/bash

#makeVCF.bash

#select from the vcf with chromosomes indicated as numbers, for plinkAnalyses
#filter on MAF > 0.01
#nb alleles max = 2

module load -f /home/gencel/vignal/save/000_ProgramModules/program_module

#Select the 403 samples and filter on MAF001

bcftools view  --samples-file /home/agirardon/work/seqapipopOnHAV3_1/RFmix/IndsAll403.list \
            --min-af 0.01:minor \
            --max-alleles 2 \
            --output-file /home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApipop_403_MAF001_diAllelic_plink.vcf.gz \
            --output-type z \
            /home/agirardon/work/seqapipopOnHAV3_1/RFmix/rename.vcf.gz
bcftools index /home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApipop_403_MAF001_diAllelic_plink.vcf.gz 

```

### VCFs par chromosome

- Shapeit requiert un VCF par chromosome.

En effet, Shapeit a besoin d'un ordre ascendant pour les positions des SNPs, et dans le VCF avec tous les chromosomes les positions sont dans l'ordre ascendant pour un chromosome, mais quand on passe au chromosome suivant la position revient au début de ce chromosome 

```bash
#!/bin/bash

#separateChromosomes.sh

module load -f /home/gencel/vignal/save/000_ProgramModules/program_module


#Select the 403 samples and filter on MAF001

for i in  $(seq 1 16)

do

bcftools view  --regions ${i} \
        --output-file /home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApipop_403_MAF001_diAllelic_plink_chr${i}.vcf.gz \
       /home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApipop_403_MAF001_diAllelic_plink.vcf.gz
bcftools index -f /home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApipop_403_MAF001_diAllelic_plink_chr${i}.vcf.gz
done

for i in  $(seq 1 16)
do
bcftools index -f /home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApipop_403_MAF001_diAllelic_plink_chr${i}.vcf.gz
done
```

### Phasage avec Shapeit

#### Mise en phase

Remarque : le fichier vcf sera renommé, parce que Shapeit décompresse les fichiers, s'ils ont .gz dans leur nom. C'est pourquoi .gz est seulement enlevé du nom VCF.

```bash
#!/bin/bash

#phasingShapeit.bash

module load bioinfo/shapeit.v2.904



for i in  $(seq 1 16)

do

sbatch --mem=50g --wrap="shapeit --input-vcf SeqApipop_403_MAF001_diAllelic_plink_chr${i}.vcf --force -O SeqApiPop_403_MAF001_diAllelic_phased_chr${i}.vcf"

done
```

#### Convertir les génotypes en phase en vcf

Chaque boucle a été lancée l'une après l'autre 

```bash
#!/bin/bash

#convertToVcf.bash

module load bioinfo/shapeit.v2.904
module load bioinfo/bcftools-1.9

for i in $(seq 1 16)

do



sbatch --wrap="shapeit -convert --input-haps SeqApiPop_403_MAF001_diAllelic_phased_chr${i}.vcf --output-vcf SeqApiPop_403_MAF001_diAllelic_phased_chr${i}.vcf"

done

for i in $(seq 1 16)

do

sbatch --wrap="bgzip SeqApiPop_403_MAF001_diAllelic_phased_chr${i}.vcf"
```


#### Concaténation de VCFs 


- Faire une liste

```bash
#!/bin/sh 

ls ~/work/seqapipopOnHAV3_1/RFmix/out/*phased*.vcf.gz > vcf.list

```

Le fichier vcf.list ressemble à : 

```
/home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_403_MAF001_diAllelic_phased_chr10.vcf.gz
/home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_403_MAF001_diAllelic_phased_chr11.vcf.gz
/home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_403_MAF001_diAllelic_phased_chr12.vcf.gz

```

Les chromosomes ne sortent pas dans le bon ordre, il suffit de couper et coller pour les réordonner.

## RFMix 

### Select reference and query Samples

Select from an Admixture Q matrix with K = 3 the individuals with > 0.93 pure backgrounds as reference => IndsReference.list The other samples => IndsQuery.list

#### Make BCF files

- References


```bash
#!/bin/bash

#selectBcfRef.bash

module load bioinfo/bcftools-1.9

#Select the 403 samples and filter on MAF001

bcftools view  --samples-file ~/work/seqapipopOnHAV3_1/RFmix/IndsAll403ref.list  \
        --output-file ~/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_320_MAF001_diAllelic_phased_Ref.bcf \
        --output-type b \
        ~/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_403_MAF001_diAllelic_phased_All.vcf.gz
bcftools index SeqApiPop_320_MAF001_diAllelic_phased_Ref.bcf
```

- Query

```bash
#!/bin/bash

#selectBcfQuery.bash

module load bioinfo/bcftools-1.9

bcftools view  --samples-file ~/work/seqapipopOnHAV3_1/RFmix/IndsAll403_query.list \
            --output-file ~/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_403_MAF001_diAllelic_phased_Query.bcf \
            --output-type b \
            ~/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_629_MAF001_diAllelic_phased_All.vcf.gz
bcftools index SeqApiPop_403_MAF001_diAllelic_phased_Query.bcf
```



## Faire une carte génétique

Pour faire la carte génétique, ici nous avons simplement récupéré la même carte génétique qu'Alain Vignal a utilisé pour son analyse RFmix. 

Trouver les crossing-overs dans les données de Liu et al. 2015

Liu H, Zhang X, Huang J, Chen J-Q, Tian D, Hurst LD, et al. Causes et conséquences du crossing-over mis en évidence via un paysage recombinatoire à haute résolution de l'abeille domestique. Genome Biol. 2015 Jan 1;16:15.

Les lectures du projet SRP043350 (Liu et al., 2015) ont été extraites de Short Read Archive (SRA) (https://www.ncbi.nlm.nih.gov/sra), alignées sur le génome de référence pour la détection des SNP (pipeline GATK) : fichier vcf LiuGenotypesNPs2allelesForCoSearchDuplOK.vcf.gz.

Le script find_crossing_overs.py a ensuite été utilisé pour détecter les crossing overs :

    -l Colonie*.list : liste des échantillons de la colonie
    -q SRR* : la reine de la colonie
    -c 6 : nombre de crossing-overs autorisés simultanément sur tous les individus.

Ensuite, une carte génétique a été construite à partir de ces données :

Après avoir importé les 3 fichiers Colony3.list, faire un tableau avec une ligne lorsque plusieurs COs à une position et ajouter une colonne indiquant les lignes qui sont impliquées dans les doubles COs.

   - Un double CO est dû soit à :
        - un événement de non-crossing-over marqué par un seul SNP
        - Les événements de non-crossing-over marqués par plus d'un SNP seront détectés par la distance au CO précédent et/ou au CO suivant chez le même individu. La distance est fixée ici à 10 kb, comme suggéré par Liu, et al., 2015.
        - Une erreur de génotypage (notamment dans le cas où nb_COs = 1 ou 2)
        - nb_COs = nombre de descendants d'une colonie, avec des COs à la même position.
        - Problèmes dans l'assemblage, comme de petites inversions, surtout dans le cas de nb_COs > 2)
   - Détection des doubles crossing-overs : deux lignes doivent être retirées
       - Ligne pour le recombinant avant le SNP : le vecteur précédent et le suivant sont identiques.
       - Ligne pour le recombinant après le SNP : le vecteur actuel examiné et les vecteurs deux pas en arrière sont identiques.



## Exécuter RFmix

Ajouter les colonnes de population à IndsAll403ref.list -> IndsAll403popref.list



```
head IndsAll403popref

Ab-PacBio	Black
BER10	Yellow
BER11	Yellow
BER12	Yellow
BER13	Yellow
BER14	Yellow
BER15	Yellow
BER16	Yellow
BER18	Yellow
BER19	Yellow
```

Nous créons donc un script pour exécuter le script RFmix : LanceRunRFmix.sh
    
```bash
#!/bin/bash

#LanceRunRFMix.bash

for i in $(seq 1 16)
do
sbatch --mem=30g ~/work/seqapipopOnHAV3_1/RFmix/runRFMixWithGenetMap.bash ${i}

done
```


This allows us to run the following script, launching the RFmix on all our chromosomes : runRFMixWithGenetMap.bash



```bash

#!/bin/bash

module load bioinfo/rfmix-9505bfa
module load bioinfo/bcftools-1.9

rfmix -f ~/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_403_MAF001_diAllelic_phased_Query.bcf \
        -r ~/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_320_MAF001_diAllelic_phased_Ref.bcf \
        --chromosome=${1} \
        -m ~/work/seqapipopOnHAV3_1/RFmix/IndsAll403popref.list \
        -g ~/work/seqapipopOnHAV3_1/RFmix/GenetMap_march_2021_AV.txt \
        -o ~/work/seqapipopOnHAV3_1/RFmix/out2/SeqApiPopRFmixChr${1}
        --reanalyze-reference 
    
```


Exemple d'output :
    
- SeqApiPopRFmixChr10.fb.tsv
- SeqApiPopRFmixChr10.msp.tsv
- SeqApiPopRFmixChr10.rfmix.Q
- SeqApiPopRFmixChr10.sis.tsv